In [ ]:
!pip install -Uqq fastai

In [ ]:
from fastai import *
from fastai.text.all import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, Markdown

In [ ]:
# Download the data
path = untar_data(URLs.IMDB)
list(path.ls())

In [ ]:
get_imdb = partial(get_text_files, folders=['train', 'unsup'])

lm_dls = DataBlock(
    blocks=TextBlock.from_folder(path, is_lm=True),
    get_items=get_imdb, splitter=RandomSplitter(0.1)
).dataloaders(path, path=path)

In [ ]:
lm_dls.show_batch(max_n=2)

In [ ]:
x, y = lm_dls.one_batch()

In [ ]:
x[0]

In [ ]:
y[0]

In [ ]:
# Check that the same shift occurs for the entire batch
assert (x[:, 1:] == y[:, :-1]).all()

In [ ]:
lm_learn = language_model_learner(lm_dls, AWD_LSTM, pretrained=True)

In [ ]:
lm_learn.model

In [ ]:
# The model is trained on wikitext, so it will be able to
# generate semi-sensible sentences, but they will likely
# have nothing to do with movies.
lm_learn.predict('This movie was terrible, ', n_words=10)

In [ ]:
LR = 1e-3

In [ ]:
lm_learn.fit_one_cycle(1, LR)

In [ ]:
lm_learn.predict('This movie was terrible, ', n_words=10)

In [ ]:
# Save the model
lm_learn.save_encoder('lm_encoder')

In [ ]:
# Create dataloaders for text classification.
# Let's make sure to pass our vocab from our pre-training
# so the word indices are the same.
dls = TextDataLoaders.from_folder(path, train='train', valid='test', text_vocab=lm_dls.vocab)

In [ ]:
dls.show_batch(max_n=3)

In [ ]:
cbs = [
    EarlyStoppingCallback(),
    SaveModelCallback()
]

In [ ]:
# Create a learner for text classification.
learn = text_classifier_learner(dls, AWD_LSTM, metrics=[accuracy, error_rate], cbs=cbs)

In [ ]:
# Load our weights from fine-tuning on full IMDB
learn = learn.load_encoder('lm_encoder')

In [ ]:
learn.fine_tune(10, 1e-3)

In [ ]:
learn.predict('This movie was horrible!')

In [ ]:
learn.predict('This movie was great!')

In [ ]:
learn.save('clf_fine_tuned')

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)

In [ ]:
interp.plot_confusion_matrix()

In [ ]:
interp.plot_top_losses(10)

In [ ]:
cbs = [
    EarlyStoppingCallback(),
    SaveModelCallback()
]

In [ ]:
control = text_classifier_learner(dls, AWD_LSTM, pretrained=False, metrics=[accuracy, error_rate], cbs=cbs).to_fp16()

In [ ]:
control.fine_tune(2, base_lr=1e-3)

In [ ]:
cbs = [
    EarlyStoppingCallback(),
    SaveModelCallback()
]

In [ ]:
control = text_classifier_learner(dls, AWD_LSTM, pretrained=True, metrics=[accuracy, error_rate], cbs=cbs).to_fp16()

In [ ]:
control.fine_tune(2, base_lr=1e-3)

# Exercise 17.1

In this exercise, you will perform an experiment of your own.
We want to know how much of a head-start starting with a pre-trained model gives us in our fine-tuning step.
In this exercise, train two language models - one that is pre-trained, and one that is not - on the `train` and `unsup` directories.
Feel free to use the `lm_dls` we defined earlier, or practice creating your own.
After the same number of epochs, is the loss lower with the pre-trained model?

<!-- startquestion -->

In [ ]:
# Your code here